In [13]:
import requests
from bs4 import BeautifulSoup
import lxml.html as lh
import pandas as pd
import re
import csv


URL = requests.get("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population")
soup = BeautifulSoup(URL.text, 'lxml')
tables = soup.find_all('table',{'class':'wikitable sortable'})
tempcols=[]
for th in tables[0].find_all('th'):
    x=th.text.replace("\n", "").replace("[c]","")
    tempcols.append(x)
cols=[]
#Fetch and clean headers
for th in tempcols:
    if th=='2016 land area':
        th = re.sub(r"(\d+)\B(\D+)","\\1 \\2", str(th))
        cols.append(th+" sq mi")
        cols.append(th+" sq km")
    elif th=='2016 population density':
        th = re.sub(r"(\d+)\B(\D+)","\\1 \\2", str(th))
        cols.append(th+" per sq mi")
        cols.append(th+" per sq km")
    elif th=='Location':
        cols.append('Location : Latitude / Longitude / Coordinates')
    else:
        th = re.sub(r"(\d+)\B(\D+)","\\1 \\2", str(th))
        cols.append(th)
        
rows=[]
#tables list contains all the tables in the given link, and each of them are stored in each index of the list
#First list contains the first table which is our main table to extract data from
rows=tables[0].find_all('tr')
finaldata=[]
for tr in rows[1:]:
    dataperrow = tr.find_all('td')
    cdata=[]
    for td in dataperrow:
        x = td.text
        x = re.sub(r'\[\w+\]', '', str(x))
        x = re.sub(r"(\d+)\B(\D+)","\\1 \\2", str(x))
        cdata.append(x.replace("\n",""))
    finaldata.append(cdata)
    
#Get wikipedia link for each individual city
citylinks = []
for tr in rows[1:]:
    citylinks.append('https://en.wikipedia.org' + str(tr.find('a',attrs={'href': re.compile("wiki/")}).get('href')))

#Store the final data in the table in a dataframe    
df=pd.DataFrame(finaldata,columns=cols)
citysummary=[]

#Iterate through links for individual cities and extract Summaries from them
for city in citylinks:
    cityurl = requests.get(city).content
    citysoup = BeautifulSoup(cityurl)
    if soup.find_all('p')[1].text != '\n':
        citysummary.append(citysoup.find_all('p')[1].text.replace("\n",""))
    else:
        citysummary.append(citysoup.find_all('p')[2].text.replace("\n",""))
        
#Append citylinks and citysummary lists to the data frame
df['Wikipedia Link'] = citylinks
df['City Summary'] = citysummary
print(df)
path=r'citydata.csv'
df.to_csv(path, index=False, encoding='UTF-8')

C:\Users\Dell\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Dell\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


    2018 rank              City                  State 2018 estimate  \
0           1     New York City               New York     8,398,748   
1           2       Los Angeles             California     3,990,456   
2           3           Chicago               Illinois     2,705,994   
3           4           Houston                  Texas     2,325,502   
4           5           Phoenix                Arizona     1,660,272   
5           6      Philadelphia           Pennsylvania     1,584,138   
6           7       San Antonio                  Texas     1,532,233   
7           8         San Diego             California     1,425,976   
8           9            Dallas                  Texas     1,345,047   
9          10          San Jose             California     1,030,119   
10         11            Austin                  Texas       964,254   
11         12      Jacksonville                Florida       903,889   
12         13        Fort Worth                  Texas       895